In [6]:
remove_words = ["@handle", "RT", "http"]

def pre_process(sentence, max_length):
    sentence = sentence.split()
    target_remove = set()
    for token in sentence:
        for target in remove_words:
            if (target == "http") and (target in token.lower()):
                target_remove.add(token)
                break
            if target in token:
                target_remove.add(token)
                break  
    for target in target_remove:
        while target in sentence:
            sentence.remove(target)
    max_length = max(max_length, len(sentence))
    sentence = ' '.join(sentence)
    return sentence, max_length


In [7]:
import csv
import collections
import numpy as np
import pandas as pd

train_file_path = "data/train_tweets.txt"
train_dict = collections.defaultdict(list)
max_length = 0

length_array = []
with open(train_file_path, encoding='utf-8') as tsvfile:
    reader = tsvfile.readlines()
    for i, row in enumerate(reader):
        row = row.strip().split("\t")
        id = int(row[0])
        instance, max_length = pre_process(row[1], max_length)
        if not instance == "":
            train_dict[id].append(instance)
            length_array.append(len(instance.split()))
    print("Total rows: %d" % i)
    
print("Total ids: %d" % len(train_dict))
print("Longest Sentence: %d" % (max_length))


Total rows: 328931
Total ids: 9295
Longest Sentence: 37


In [8]:
import random
import numpy as np

dev_split = 0.1
train_split = 1 - dev_split

dev_set_dict = {}
train_set_dict = {}

for id in train_dict:
    target_list = train_dict[id]
    length = len(target_list)
    random.shuffle(target_list)
    split = int(np.ceil(length*dev_split))
    dev_set_dict[id] = target_list[:split]
    train_set_dict[id] = target_list[split:length]
#     print(len(dev_set_dict[id]), len(train_set_dict[id]), length)

print(len(dev_set_dict), len(train_set_dict), len(train_dict))


9295 9295 9295


In [22]:
import pickle
import pandas as pd

a = None
def save_to_file(target_dict, file_path):
    id_list = []
    sentence_list = []
    for id in target_dict:
        all_sentences =  target_dict[id]
        id_list.append(id)
        sentence_list.append(all_sentences)
      
    id_list = np.array(id_list)
    sentence_list = np.array(sentence_list)
    random_index = np.array(range(len(sentence_list)))
    random.shuffle(random_index)
    id_list = id_list[random_index]
    sentence_list = sentence_list[random_index]
    
    dataframe = pd.DataFrame({'id':id_list,'sentence':sentence_list})
    dataframe.to_pickle(file_path)
    #dataframe.to_csv(file_path,index=False,sep='\t',header=None)
    print(len(id_list))
    return

version = "v1_2"
version_name = "{0}_one_user_one_snippet".format(version)
dev_set_path = 'data/{0}/dev_set_{1}.txt'.format(version, version_name)
train_set_path = 'data/{0}/train_set_{1}.txt'.format(version, version_name)
idx_file_path = 'data/{0}/{1}_idx.pickle'.format(version, version_name)

save_to_file(dev_set_dict, dev_set_path)
save_to_file(train_set_dict, train_set_path)


# Build IDX and Save
idx = {}
for i, id in enumerate(train_dict):
    idx[id] = i
print(len(idx))

with open(idx_file_path, 'wb') as handle:
    pickle.dump(idx, handle, protocol=pickle.HIGHEST_PROTOCOL)


9295
9295
9295
